<a href="https://colab.research.google.com/github/shielamms/Machine-Learning-Notebooks/blob/master/IntermediateML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Handling Missing Values

In [0]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [0]:
X_train_full = pd.read_csv('home-data-for-ml-course/train.csv', index_col='Id')
X_test_full = pd.read_csv('home-data-for-ml-course/test.csv', index_col='Id')

In [0]:
X_train_full.head(3)

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
Id,,,,,,,,,,,,,,,,,,,,,
1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500


In [0]:
# Remove data with blank SalePrice
X_train_full.dropna(axis=0, subset=['SalePrice'], inplace=True)
y = X_train_full['SalePrice']
X_train_full.drop(['SalePrice'], axis=1, inplace=True)

X = X_train_full.select_dtypes(exclude=['object'])
X_test = X_test_full.select_dtypes(exclude=['object'])

In [0]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2, random_state=0)

In [0]:
X_train.head()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold
Id,,,,,,,,,,,,,,,,,,,,,
619,20,90.0,11694,9,5,2007,2007,452.0,48,0,...,774,0,108,0,0,260,0,0,7,2007
871,20,60.0,6600,5,5,1962,1962,0.0,0,0,...,308,0,0,0,0,0,0,0,8,2009
93,30,80.0,13360,5,7,1921,2006,0.0,713,0,...,432,0,0,44,0,0,0,0,8,2009
818,20,NaN,13265,8,5,2002,2002,148.0,1218,0,...,857,150,59,0,0,0,0,0,7,2008
303,20,118.0,13704,7,5,2001,2002,150.0,0,0,...,843,468,81,0,0,0,0,0,1,2006


In [0]:
missing_val_count_by_column = (X_train.isnull().sum())
print(missing_val_count_by_column[missing_val_count_by_column > 0])

LotFrontage    212
MasVnrArea       6
GarageYrBlt     58
dtype: int64


In [0]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

def score_dataset(X_train, X_valid, y_train, y_valid):
    model = RandomForestRegressor(n_estimators=100, random_state=0)
    model.fit(X_train, y_train)
    preds = model.predict(X_valid)
    return mean_absolute_error(y_valid, preds)

In [0]:
# Method 1: Dropping columns
cols_with_missing_values = [col for col in X_train.columns
                           if X_train[col].isnull().any()]

reduced_X_train = X_train.drop(cols_with_missing_values, axis=1)
reduced_X_valid = X_valid.drop(cols_with_missing_values, axis=1)

drop_score = score_dataset(reduced_X_train, reduced_X_valid, y_train, y_valid)
drop_score

17837.82570776256

In [0]:
# Method 2: Imputation
from sklearn.impute import SimpleImputer

imputer = SimpleImputer()

X_train_imputed = pd.DataFrame(imputer.fit_transform(X_train))
X_valid_imputed = pd.DataFrame(imputer.transform(X_valid))

X_train_imputed.columns = X_train.columns
X_valid_imputed.columns = X_valid.columns

impute_score = score_dataset(X_train_imputed, X_valid_imputed, y_train, y_valid)
impute_score

18062.894611872147

In [0]:
# Decision: Use imputation
final_imputer = SimpleImputer(strategy='median')

final_X_train = pd.DataFrame(final_imputer.fit_transform(X_train))
final_X_valid = pd.DataFrame(final_imputer.transform(X_valid))

final_X_train.columns = X_train.columns
final_X_valid.columns = X_valid.columns

model = RandomForestRegressor(n_estimators=100, random_state=0)
model.fit(final_X_train, y_train)

predictions_valid = model.predict(final_X_valid)
print(mean_absolute_error(y_valid, predictions_valid))

17791.59899543379


In [0]:
# Test the model on the test data
final_X_test = pd.DataFrame(final_imputer.transform(X_test))
final_X_test.columns = X_test.columns

test_predictions = model.predict(final_X_test)
test_predictions

array([125985.5 , 154599.5 , 180070.24, ..., 154751.44, 107387.  ,
       229281.55])

In [0]:
# Output predictions
output = pd.DataFrame({'Id': final_X_test.index,
                     'SalePrice': test_predictions})
output.to_csv('submission.csv', index=False)